In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

from util import load_data, fdr, plot_report

%matplotlib inline

In [2]:
x_train, x_test, x_oot, y_train, y_test, y_oot = load_data('var_30.csv')
x_train.head()

,ssn_name_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,ssn_firstname_count_14,ssn_dob_count_0_by_30,name_dob_count_0_by_30,fulladdress_homephone_count_14,fulladdress_count_1_by_14,fulladdress_homephone_day_since,ssn_dob_day_since,...,fulladdress_count_0,name_dob_count_3,fulladdress_homephone_count_0_by_14,name_day_since,name_count_14,ssn_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_firstname_count_0_by_14,ssn_name_day_since,address_count_1_by_7
409608,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.142853,-0.149881,...,-0.041351,-0.046415,0.120387,0.070807,-0.159376,0.118004,0.123458,0.123528,-0.132851,0.103441
722045,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,0.939417,0.931715,...,-0.041351,-0.046415,0.120387,1.148480,-0.159376,0.118004,0.123458,0.123528,0.951197,0.103441
294840,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.541584,-0.548364,...,-0.041351,-0.046415,0.120387,-0.326230,-0.159376,0.118004,0.123458,0.123528,-0.532238,0.103441
59184,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-1.367527,-1.373793,...,-0.041351,-0.046415,0.120387,-1.148664,-0.159376,0.118004,0.123458,0.123528,-1.359538,0.103441
163866,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.997277,-1.003773,...,-0.041351,-0.046415,0.120387,-0.779986,-0.159376,0.118004,0.123458,0.123528,-0.988679,0.103441


In [3]:
lr = LogisticRegression(max_iter=1000)
params = {'C': [0.001, 0.01, 0.1, 1, 10],
          'class_weight': ['balanced', None]}
clf = GridSearchCV(lr, params, cv=5, refit='FDR', verbose=1,
                   n_jobs=8, scoring={'FDR': fdr, 'ROC': 'roc_auc'})
clf.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:  4.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'class_weight': ['balanced', None]},
             pre_dispatch='2*n_jobs', refit='FDR', return_train_score=False,
             scoring={'FDR': <function fdr at 0x7ff784c5d510>,
                      'ROC': 'roc_auc'},
             verbose=1)

In [4]:
print(f"The best parameters are {clf.best_params_} "
      f"with a score of {clf.best_score_:0.5f}")

The best parameters are {'C': 10, 'class_weight': 'balanced'} with a score of 0.50225


In [5]:
pd.DataFrame(clf.cv_results_).filter(
    items=['mean_fit_time', 'mean_score_time',
           'params', 'mean_test_FDR','std_test_FDR',
           'rank_test_FDR', 'mean_test_ROC', 'std_test_ROC'])

,mean_fit_time,mean_score_time,params,mean_test_FDR,std_test_FDR,rank_test_FDR,mean_test_ROC,std_test_ROC
0,9.930253,1.008337,"{'C': 0.001, 'class_weight': 'balanced'}",0.496216,0.010396,5,0.755338,0.006150
1,4.693320,1.065794,"{'C': 0.001, 'class_weight': None}",0.487108,0.010716,10,0.748179,0.007319
2,20.797694,1.257962,"{'C': 0.01, 'class_weight': 'balanced'}",0.500934,0.010636,4,0.753614,0.006239
3,11.712570,1.339132,"{'C': 0.01, 'class_weight': None}",0.487327,0.011227,9,0.748516,0.007001
4,42.191806,1.222250,"{'C': 0.1, 'class_weight': 'balanced'}",0.501483,0.011168,3,0.753575,0.006053
5,26.505790,1.258205,"{'C': 0.1, 'class_weight': None}",0.491168,0.010901,8,0.748517,0.006759
6,65.553280,1.169910,"{'C': 1, 'class_weight': 'balanced'}",0.502141,0.011285,2,0.753443,0.006083
7,48.079063,1.227704,"{'C': 1, 'class_weight': None}",0.491935,0.011368,7,0.748346,0.006764
8,66.157974,1.149063,"{'C': 10, 'class_weight': 'balanced'}",0.502251,0.011433,1,0.753469,0.006080
9,42.200982,0.895401,"{'C': 10, 'class_weight': None}",0.491936,0.011449,6,0.748258,0.006777


In [10]:
lr = LogisticRegression(max_iter=1000, C=1, class_weight='balanced').fit(x_train, y_train)
print(fdr(lr, x_train, y_train))
print(fdr(lr, x_test, y_test))
print(fdr(lr, x_oot, y_oot))

0.5026884670251289
0.4874947011445528
0.500513522766176


In [11]:
x_train, x_test, x_oot, y_train, y_test, y_oot = load_data('var_40.csv')
lr = LogisticRegression(max_iter=1000, C=1, class_weight='balanced').fit(x_train, y_train)
print(fdr(lr, x_train, y_train))
print(fdr(lr, x_test, y_test))
print(fdr(lr, x_oot, y_oot))

0.5334424440807419
0.5387949718248808
0.5282437521396782
